In [ ]:
### This script calculates transport SigmaZ plots, AMOC streamfunctions and timeseries, 
### as well as SigmaZ diagrams for the MHT and MFT fields, and timeseries of MHT and MFT components. 
### Applied to all CM4 control runs

# First apply the SigmaZ routines to model data:

In [ ]:
%matplotlib inline
import sys
sys.path.insert(0, '/home/Matthew.Thomas/python_code/python_pkgs/SigmaZ_diag/SigmaZ_diag')

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
import cartopy.crs as ccrs
from numba import njit
import sectionate
from glob import glob
import momlevel
import SigmaZ_funcs
from xhistogram.xarray import histogram
import xrft

import warnings
warnings.filterwarnings('ignore')

## Setup a dask server if needed:

In [ ]:
# import dask
# from dask.distributed import Client
# from dask_jobqueue import SLURMCluster
# dask.config.set(**{'array.slicing.split_large_chunks': False})

# portdash = 18234

# cluster = SLURMCluster(
#     queue="batch",
#     cores=8,
#     processes=4,
#     project="gfdl_o",
#     memory="48GB",
#     walltime="6:00:00",
#     local_directory="$TMPDIR",
#     death_timeout=120,
#     scheduler_options={"dashboard_address":f":{portdash}"},
# )

# client = Client(cluster)
# client

In [ ]:
# cluster.scale(jobs=10)

## Dictionary of all cross sections to run the script on:

In [ ]:
sections_dict = {
                 'lat' : {'sect1' : tuple([52.0166, 52.6648, 53.5577, 58.8944, 60.4000]),
                          'sect2' : tuple([60.3000, 58.8600, 58.0500, 58.0000, 56.5000])                          
                          },
                 'lon' : {'sect1' : tuple([-56.8775, -52.0956, -49.8604, -47.6107, -44.8000]),
                          'sect2' : tuple([-44.9000, -30.5400, -28.0000, -14.7000, -5.9300])
                          }
                 }

## dictionary of all input variables for the different model runs to look at:

In [ ]:
run_dict = {
                'odiv209' : {
                             'vars_path' : "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20210706/CM4_piControl_c192_OM4p125_v7/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_month_z/ts/monthly/5yr/",
                             'grid_path' : "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20210706/CM4_piControl_c192_OM4p125_v7/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_month_z/ocean_month_z.static.nc",
                             'file_str_identifier' : "*", # This is a linux-style search modifier for selecting files. The function will search for vars_path/file_str_identifier. E.g.: *.0[1-2]*.nc will search for /dir/of/vars/*.0[1-2]*.nc (all files from year 100 to 200)
                             'z_layer_var' : "z_l",
                             'z_inter_var' : "z_i", 
                             'color_identifier' : 'b',  # This is the color used for line plots 
                             },
                'odiv230' : {
                             'vars_path' : "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20221223/CM4_piControl_c192_OM4p25_v8/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_monthly_z/ts/monthly/5yr/",
                             'grid_path' : "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20221223/CM4_piControl_c192_OM4p25_v8/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_monthly_z/ocean_monthly_z.static.nc",
                             'file_str_identifier' : "*", 
                             'z_layer_var' : "z_l",
                             'z_inter_var' : "z_i",
                             'color_identifier' : 'yellow',
                             },
                'RDC_p25' : {
                             'vars_path' : "/archive/roz/cm2.hi/CM2.hi_Control_1990_E01_rs_wind_cars_woa_newtopog/gfdl.ncrc3-intel15-prod/pp/ocean/ts/monthly/5yr/",
                             'grid_path' : "/archive/roz/cm2.hi/CM2.hi_Control_1990_E01_rs_wind_cars_woa_newtopog/gfdl.ncrc3-intel15-prod/pp/ocean/ocean.static.nc",
                             'file_str_identifier' : "*.0076*",
                             'mom_gridwidth_algo' : False,
                             'z_layer_var' : "st_ocean",
                             'z_inter_var' : "st_edges_ocean",
                             'u_transport_var' : "tx_trans",
                             'v_transport_var' : "ty_trans",
                             'theta_var' : "temp",
                             'salt_var' : "salt",
                             'x_hpoint_1Dvar' : "xt_ocean",
                             'x_qpoint_1Dvar' : "xu_ocean",
                             'y_hpoint_1Dvar' : "yt_ocean",
                             'y_qpoint_1Dvar' : "yu_ocean",
                             'lons_tpoint' : "geolon_t",
                             'lats_tpoint' : "geolat_t",
                             'lons_cpoint' : "geolon_c",
                             'lats_cpoint' : "geolat_c",
                             'color_identifier' : 'purple',   
                             'min_sigma_cutoff' : 26.5,
                             },
                'odiv1' :   {
                             'vars_path' : "/archive/oar.gfdl.cmip6/CM4/warsaw_201710_om4_v1.0.1/CM4_piControl_C/gfdl.ncrc4-intel16-prod-openmp/pp/ocean_monthly_z/ts/monthly/5yr/",
                             'grid_path' : "/archive/oar.gfdl.cmip6/CM4/warsaw_201710_om4_v1.0.1/CM4_piControl_C/gfdl.ncrc4-intel16-prod-openmp/pp/ocean_monthly_z/ocean_monthly_z.static.nc",
                             'mom_gridwidth_algo' : False,
                             'file_str_identifier' : "*.0[0-2]*",
                             'z_layer_var' : "z_l",
                             'z_inter_var' : "z_i",
                             'color_identifier' : 'r',
                             'color_identifier' : 'pink',
                             },
                # # # '1deg_DA' : {
                # # #              'vars_path' : "/archive/Feiyu.Lu/SPEAR/SPEAR_c96_o1_ECDA_J11/pp_ensemble/ocean_z/ts/monthly/30yr/",
                # # #              'grid_path' : "/archive/Feiyu.Lu/SPEAR/SPEAR_c96_o1_ECDA_J11/pp_ensemble/ocean_z/ocean_z.static.nc",
                # # #              'mom_gridwidth_algo' : False,
                # # #              'z_layer_var' : "z_l",
                # # #              'z_inter_var' : "z_i",
                # # #              'time_limits' : ['2014','2020'], #use this date style if 'decode_times_flag'=True
                # # #              'file_str_identifier' : "*",
                # # #              'color_identifier' : 'brown',
                # # #              },
}

## Load the data according to the input variables listed in the run_dict:

In [ ]:
###  A few necessary variables:
rholims=[21,28.1]; # Density range for the rebinning into the SigmaZ diagram
nrho=500;  # number of density bins between the two rholims
rebin_depth=np.arange(100,6600,100) # regularly spaced rebinned z-layers for the SigmaZ diagram

In [ ]:
%%time

dmgetout_flag = False
zarr_dir='/xtmp/Matthew.Thomas/work'

ComputeDiag_dict = {}
if dmgetout_flag: print(f'run the following dmget commands:')
for section in sections_dict['lat']:
    ComputeDiag_dict[section]= {}
    for keys in run_dict:
        var_dict = {'numpy_algo' : False , 'mom_gridwidth_algo' : True , 'z_layer_var' : "z_l", 'z_inter_var' : "z_i" , 'plot_flag' : False, 'decode_times_flag' : True,  'load_var' : True, 'time_limits' : [], 'rho_var'  : "", 'salt_var' : 'so' , 'theta_var' : 'thetao' , 'years_choice_for_SigmaZ' : [],
                    'u_transport_var' : "umo" , 'v_transport_var' : "vmo" , 'x_hpoint_1Dvar' : "xh" , 'x_qpoint_1Dvar' : "xq" , 'y_hpoint_1Dvar' : "yh" , 'y_qpoint_1Dvar' : "yq", 'lons_tpoint' : "geolon" , 'lats_tpoint' : "geolat" , 'lons_cpoint' : "geolon_c" , 'lats_cpoint' : "geolat_c", 'rho0' : 1030 , 'supergrid_flag' : False, 'calc_transp' : False, 'dir_deptho' : '', 'min_sigma_cutoff' : 26.5}

        for key in run_dict[keys]: var_dict[key]=run_dict[keys][key]

        dsT, transp, rho, xsec, ysec, cell_area, so, thetao, section_gridwidth, grid = SigmaZ_funcs.SigmaZ_diag_ReadData(var_dict['vars_path'],var_dict['grid_path'],sections_dict['lon'][section],sections_dict['lat'][section],var_dict['file_str_identifier'],rho_var=var_dict['rho_var'],theta_var=var_dict['theta_var'],salt_var=var_dict['salt_var'],z_layer_var=var_dict['z_layer_var'],z_inter_var=var_dict['z_inter_var'],u_transport_var=var_dict['u_transport_var'],v_transport_var=var_dict['v_transport_var'],time_limits=var_dict['time_limits'],x_hpoint_1Dvar=var_dict['x_hpoint_1Dvar'],x_qpoint_1Dvar=var_dict['x_qpoint_1Dvar'],y_hpoint_1Dvar=var_dict['y_hpoint_1Dvar'],y_qpoint_1Dvar=var_dict['y_qpoint_1Dvar'],lons_tpoint=var_dict['lons_tpoint'],lats_tpoint=var_dict['lats_tpoint'],lons_cpoint=var_dict['lons_cpoint'],lats_cpoint=var_dict['lats_cpoint'],dir_deptho=var_dict['dir_deptho'],mom_gridwidth_algo=var_dict['mom_gridwidth_algo'],decode_times_flag=var_dict['decode_times_flag'],supergrid_flag=var_dict['supergrid_flag'],calc_transp=var_dict['calc_transp'],dmgetout=dmgetout_flag,zarr_dir=zarr_dir)

        if len(dsT)>0:
            depth=dsT[var_dict['z_layer_var']]
            
            ### Load data into memory: can be commented out, but it makes things faster when the output is used in multiple different operations (which it is)
            if var_dict['load_var']:
                if var_dict['numpy_algo']:
                    ### convert xarray output to numpy values (Ensure that matrix dimensions are (time,depth,dist))
                    transp=transp.values
                    rho=rho.values
                    cell_area=np.repeat(np.expand_dims(cell_area.values,axis=0),np.shape(dsT.time)[0],axis=0)
                    cell_area[np.isnan(cell_area)]=0
                    depth=depth.values
                    if var_dict['salt_var']: so=so.values
                    if var_dict['theta_var']: thetao=thetao.values
                else:
                    transp=transp.load()
                    rho=rho.load()
                    cell_area=cell_area.load()
                    if var_dict['salt_var']: so=so.load()
                    if var_dict['theta_var']: thetao=thetao.load()
                        
            ty_z_rho, ty_z_rho_rebin, rho0_bounds, rho0_ref = SigmaZ_funcs.SigmaZ_diag_ComputeDiag(transp,rho,rholims,depth,rebin_depth,nrho,numpy_algo=var_dict['numpy_algo'],z_layer_var=var_dict['z_layer_var'])

            AMOCz, AMOCrho, ty_z, ty_rho, AMOCz_ts, AMOCrho_ts = SigmaZ_funcs.SigmaZ_diag_PlotDiag(ty_z_rho_rebin,rebin_depth,rho0_ref,rho0_bounds,rho0=var_dict['rho0'],min_sigma_cutoff=var_dict['min_sigma_cutoff'],numpy_algo=var_dict['numpy_algo'],plot_flag=var_dict['plot_flag'])
            
            if var_dict['theta_var']:
                MOCsig_MHTMFT, MOCz_MHTMFT, MOCsigz_MHTMFT = SigmaZ_funcs.SigmaZ_diag_computeMFTMHT(transp,so,thetao,rho,cell_area,rho0_bounds,depth,rebin_depth=rebin_depth,rho_dim='rho_bin',z_dim=var_dict['z_layer_var'],rho0=var_dict['rho0'],annual_mean_flag=False,plot_flag=var_dict['plot_flag'])

            ComputeDiag_dict[section][keys]={'dsT' : dsT , 'transp' : transp, 'rho' : rho , 'xsec' : xsec , 'ysec' : ysec , 'cell_area' : cell_area , 'so' : so , 'thetao' : thetao , 'section_gridwidth' : section_gridwidth , 'grid' : grid,
                                             'ty_z_rho' : ty_z_rho , 'ty_z_rho_rebin' : ty_z_rho_rebin, 'rho0_bounds' : rho0_bounds , 'rho0_ref' : rho0_ref,
                                             'AMOCz' : AMOCz , 'AMOCrho' : AMOCrho, 'ty_z' : ty_z , 'ty_rho' : ty_rho , 'AMOCz_ts' : AMOCz_ts , 'AMOCrho_ts' : AMOCrho_ts,
                                             'MOCsig_MHTMFT' : MOCsig_MHTMFT , 'MOCz_MHTMFT' : MOCz_MHTMFT , 'MOCsigz_MHTMFT' : MOCsigz_MHTMFT}        
            
            print("Finished with "+keys)

In [ ]:
fig = plt.figure(figsize=(20,8))
loopnum=0
for section in sections_dict['lat']:
    fig.add_subplot(1,len(sections_dict['lat'])+1,loopnum+1)
    for keys in run_dict:
        transport_scale = 1030*1e6 if abs(ComputeDiag_dict[section][keys]['AMOCrho'].mean('time')).max() > 1e8 else 1
        (ComputeDiag_dict[section][keys]['AMOCrho'].mean('time')/transport_scale).plot(label=keys,y='rho_bin',ylim=[28,25],yincrease=False,color=run_dict[keys]['color_identifier'])
    loopnum+=1
    plt.legend()
if loopnum>0:
    if var_dict['numpy_algo']==False:
        fig.add_subplot(1,len(sections_dict['lat'])+1,loopnum+1)
        for keys in run_dict:
            section_tot=0
            for section in sections_dict['lat']:
                transport_scale = 1030*1e6 if ComputeDiag_dict[section][keys]['AMOCrho'].mean('time').max() > 1e8 else 1
                section_tot+=ComputeDiag_dict[section][keys]['AMOCrho']
            (section_tot.mean(dim='time')/transport_scale).plot(label=keys,y='rho_bin',ylim=[28,25],yincrease=False,color=run_dict[keys]['color_identifier'])
        plt.legend()

In [ ]:
smoothing_level=10 # in years
fig = plt.figure(figsize=(25,8))
loopnum=0
for section in sections_dict['lat']:
    fig.add_subplot(1,len(sections_dict['lat'])+1,loopnum+1)
    for keys in run_dict:
        transport_scale = 1030*1e6 if ComputeDiag_dict[section][keys]['AMOCrho_ts'].max() > 1e8 else 1
        xsection=ComputeDiag_dict[section][keys]['AMOCrho']
        (xsection/transport_scale).where(xsection['rho_bin']>26.5).max(dim='rho_bin').plot(alpha=.7,linewidth=.5,color=run_dict[keys]['color_identifier'])
        (xsection/transport_scale).resample(time="Y").mean().where(xsection['rho_bin']>26.5).max(dim='rho_bin').rolling(time=smoothing_level,center=True).mean().plot(label=keys,color=run_dict[keys]['color_identifier'])
    loopnum+=1
    plt.xlabel('years')
    plt.legend()
if loopnum>0:
    if var_dict['numpy_algo']==False:
        fig.add_subplot(1,len(sections_dict['lat'])+1,loopnum+1)
        for keys in run_dict:
            section_tot=0
            for section in sections_dict['lat']:
                transport_scale = 1030*1e6 if ComputeDiag_dict[section][keys]['AMOCrho_ts'].max() > 1e8 else 1
                section_tot+=ComputeDiag_dict[section][keys]['AMOCrho']
            # plt.plot((section_tot.time-section_tot.time[0])/360,section_tot.rolling(time=smoothing_level,center=True).mean().max(dim='rho_bin')/transport_scale,label=keys,color=run_dict[keys]['color_identifier'])
            (section_tot/transport_scale).where(section_tot['rho_bin']>26.5).max(dim='rho_bin').plot(alpha=.7,linewidth=.5,color=run_dict[keys]['color_identifier'])
            (section_tot/transport_scale).resample(time="Y").mean().where(section_tot['rho_bin']>26.5).max(dim='rho_bin').rolling(time=smoothing_level,center=True).mean().plot(label=keys,color=run_dict[keys]['color_identifier'])
        plt.legend()
        plt.xlabel('years')

# Now use the same functions on the OSNAP observational data:

In [ ]:
### Read in OSNAP Data:
ds_OSNAP = xr.open_dataset("/home/Matthew.Thomas/work/OSNAP/OSNAP_Gridded_TSV_201408_201805_2021.nc", decode_times=True)
ds_OSNAP_streamfunction = xr.open_dataset("/work/m2t/From_Rong/OSNAP_Streamfunction_201408_201805_2020_new.nc", decode_times=True)
Area=ds_OSNAP.AREA_OSNAP
Time_OSNAP=ds_OSNAP.TIME_OSNAP
depth_OSNAP=ds_OSNAP.DEPTH_OSNAP
Transp_OSNAP=ds_OSNAP.VELO_OSNAP*Area
TEMP_OSNAP=ds_OSNAP.TEMP_OSNAP
PSAL_OSNAP=ds_OSNAP.PSAL_OSNAP
PD_OSNAP = momlevel.derived.calc_rho(TEMP_OSNAP,PSAL_OSNAP,0)-1000
earth_radius=6371  # in km
section_gridwidth_OSNAP=earth_radius*sectionate.distance_on_unit_sphere(ds_OSNAP.LAT_OSNAP[0:-1],ds_OSNAP.LON_OSNAP.values[0:-1],ds_OSNAP.LAT_OSNAP[1:],ds_OSNAP.LON_OSNAP[1:])
numpy_algo=False

In [ ]:
PD_OSNAP=PD_OSNAP.rename('rho')
if numpy_algo:
    depth_OSNAP=depth_OSNAP.values
    PD_OSNAP=PD_OSNAP.values
    TEMP_OSNAP=TEMP_OSNAP.values
    PSAL_OSNAP=PSAL_OSNAP.values
    Area=np.repeat(Area.values[np.newaxis,...],len(ds_OSNAP.TIME_OSNAP), axis=0)
    Transp_OSNAP=Transp_OSNAP.values
    section_gridwidth_OSNAP=section_gridwidth_OSNAP.values

In [ ]:
obs_dict={
          'sect1' : {},
          'sect2' : {}
          }

In [ ]:
### Calculate SigmaZ diagrams for OSNAP west
ty_z_rho, ty_z_rho_rebin, rho0_bounds, rho0_ref = SigmaZ_funcs.SigmaZ_diag_ComputeDiag(Transp_OSNAP.isel(x=slice(1,79)),PD_OSNAP.isel(x=slice(1,79)),rholims,depth_OSNAP,rebin_depth,nrho,z_layer_var='z',dist_var='x',numpy_algo=numpy_algo)
### Plot SigmaZ diagram and streamfunctions along OSNAP
AMOCz, AMOCrho, ty_z, ty_rho, AMOCz_ts, AMOCrho_ts = SigmaZ_funcs.SigmaZ_diag_PlotDiag(ty_z_rho_rebin*1030,rebin_depth,rho0_ref,rho0_bounds,numpy_algo=numpy_algo,plot_flag=False)
### Calculate Heat and Freshwater Transports along OSNAP East and West
MOCsig_MHTMFT, MOCz_MHTMFT, MOCsigz_MHTMFT = SigmaZ_funcs.SigmaZ_diag_computeMFTMHT(Transp_OSNAP.isel(x=slice(1,79))*1030,PSAL_OSNAP.isel(x=slice(1,79)),TEMP_OSNAP.isel(x=slice(1,79)),PD_OSNAP.isel(x=slice(1,79)),Area.isel(x=slice(1,79)),rho0_bounds,depth_OSNAP,rebin_depth=rebin_depth,z_dim='z',dist_var='x',plot_flag=False)
obs_dict['sect1']={'ty_z_rho' : ty_z_rho, 'ty_z_rho_rebin' : ty_z_rho_rebin, 'rho0_bounds' : rho0_bounds, 'rho0_ref' : rho0_ref,
                  'AMOCz' : AMOCz, 'AMOCrho' : AMOCrho, 'ty_z' : ty_z, 'ty_rho' : ty_rho, 'AMOCz_ts' : AMOCz_ts, 'AMOCrho_ts' : AMOCrho_ts,
                  'MOCsig_MHTMFT' : MOCsig_MHTMFT, 'MOCz_MHTMFT' : MOCz_MHTMFT, 'MOCsigz_MHTMFT' : MOCsigz_MHTMFT,
                  'PD_OSNAP' : PD_OSNAP.isel(x=slice(1,79)) , 'TEMP_OSNAP' : TEMP_OSNAP.isel(x=slice(1,79)) , 'PSAL_OSNAP' : PSAL_OSNAP.isel(x=slice(1,79)) , 'Transp_OSNAP' : Transp_OSNAP.isel(x=slice(1,79)) , 'Area' : Area.isel(x=slice(1,79)),
                  'section_gridwidth_OSNAP' : section_gridwidth_OSNAP.isel(x=slice(79,len(PD_OSNAP.x)))}

In [ ]:
### Calculate SigmaZ diagrams for OSNAP east
ty_z_rho, ty_z_rho_rebin, rho0_bounds, rho0_ref = SigmaZ_funcs.SigmaZ_diag_ComputeDiag(Transp_OSNAP.isel(x=slice(80,len(PD_OSNAP.x))),PD_OSNAP.isel(x=slice(80,len(PD_OSNAP.x))),rholims,depth_OSNAP,rebin_depth,nrho,z_layer_var='z',dist_var='x',numpy_algo=numpy_algo)
### Plot SigmaZ diagram and streamfunctions along OSNAP
AMOCz, AMOCrho, ty_z, ty_rho, AMOCz_ts, AMOCrho_ts = SigmaZ_funcs.SigmaZ_diag_PlotDiag(ty_z_rho_rebin*1030,rebin_depth,rho0_ref,rho0_bounds,numpy_algo=numpy_algo,plot_flag=False)
### Calculate Heat and Freshwater Transports along OSNAP East and West
MOCsig_MHTMFT, MOCz_MHTMFT, MOCsigz_MHTMFT = SigmaZ_funcs.SigmaZ_diag_computeMFTMHT(Transp_OSNAP.isel(x=slice(80,len(PD_OSNAP.x)))*1030,PSAL_OSNAP.isel(x=slice(80,len(PD_OSNAP.x))),TEMP_OSNAP.isel(x=slice(80,len(PD_OSNAP.x))),PD_OSNAP.isel(x=slice(80,len(PD_OSNAP.x))),Area.isel(x=slice(80,len(PD_OSNAP.x))),rho0_bounds,depth_OSNAP,rebin_depth=rebin_depth,z_dim='z',dist_var='x',plot_flag=False)
obs_dict['sect2']={'ty_z_rho' : ty_z_rho, 'ty_z_rho_rebin' : ty_z_rho_rebin, 'rho0_bounds' : rho0_bounds, 'rho0_ref' : rho0_ref,
                  'AMOCz' : AMOCz, 'AMOCrho' : AMOCrho, 'ty_z' : ty_z, 'ty_rho' : ty_rho, 'AMOCz_ts' : AMOCz_ts, 'AMOCrho_ts' : AMOCrho_ts,
                  'MOCsig_MHTMFT' : MOCsig_MHTMFT, 'MOCz_MHTMFT' : MOCz_MHTMFT, 'MOCsigz_MHTMFT' : MOCsigz_MHTMFT,
                  'PD_OSNAP' : PD_OSNAP.isel(x=slice(80,len(PD_OSNAP.x))) , 'TEMP_OSNAP' : TEMP_OSNAP.isel(x=slice(80,len(PD_OSNAP.x))) , 'PSAL_OSNAP' : PSAL_OSNAP.isel(x=slice(80,len(PD_OSNAP.x))) , 'Transp_OSNAP' : Transp_OSNAP.isel(x=slice(80,len(PD_OSNAP.x))) , 'Area' : Area.isel(x=slice(80,len(PD_OSNAP.x))),
                  'section_gridwidth_OSNAP' : section_gridwidth_OSNAP.isel(x=slice(79,len(PD_OSNAP.x)))}

# Now make some obs vs model comparison plots

In [ ]:
fig = plt.figure(figsize=(14,6))
subplot_count=0
for secs in sections_dict['lat']:
    fig.add_subplot(1,len(sections_dict['lat']),subplot_count+1)
    (obs_dict[secs]['AMOCrho'].mean('time')/1e6/1035).plot(ylim=[28,25],y='rho_bin',label='obs',color='k')
    for keys in run_dict:
        transport_scale = 1030*1e6 if ComputeDiag_dict[section][keys]['AMOCrho'].max() > 1e8 else 1
        (ComputeDiag_dict[secs][keys]['AMOCrho'].mean('time')/transport_scale).plot(ylim=[28,25],y='rho_bin',label=keys,color=run_dict[keys]['color_identifier'])
    plt.legend()
    plt.title('transport streamfunction along '+secs)
    plt.xlabel('Sv')
    subplot_count+=1

In [ ]:
smoothing_level=10   # in years
plot_variables=['MHT_sum','MHT_MOCrho_sum','MHT_GYRErho_sum']
fig = plt.figure(figsize=(16,18))
subplot_count=0
for section_choice in sections_dict['lat']:
    fig.add_subplot(len(plot_variables)+1,len(sections_dict['lat']),subplot_count+1)
    obs_MOC=(obs_dict[section_choice]['MOCsig_MHTMFT']['MOCrho']/1e6/1030).rolling(time=12).mean()
    obs_MOC['time']=obs_MOC['time']*30
    start_dates=[]
    end_dates=[]
    for keys in run_dict:
        (ComputeDiag_dict[section_choice][keys]['MOCsig_MHTMFT']['MOCrho']/1e6/1030).resample(time="Y").mean().plot(alpha=.7,linewidth=.5,color=run_dict[keys]['color_identifier'])
        (ComputeDiag_dict[section_choice][keys]['MOCsig_MHTMFT']['MOCrho']/1e6/1030).resample(time="Y").mean().rolling(time=smoothing_level, center=True).mean().plot(label=keys,color=run_dict[keys]['color_identifier'])
        if keys != 'RDC_p25': start_dates.append(ComputeDiag_dict[section_choice][keys]['MOCsig_MHTMFT'].time.values[0])
        if keys != 'RDC_p25': end_dates.append(ComputeDiag_dict[section_choice][keys]['MOCsig_MHTMFT'].time.values[-1])
    plt.title('ann. mean '+ComputeDiag_dict[section_choice][keys]['MOCsig_MHTMFT']['MOCrho'].long_name+'. '+section_choice)
    plt.ylabel('Sv')
    plt.fill_between([np.min(start_dates),np.max(end_dates)],obs_MOC.mean()-obs_MOC.std(), obs_MOC.mean()+obs_MOC.std(), alpha=0.2, label='obs (mean+/-std')
    plt.legend()
    
    loopnum=1
    for y in plot_variables:
        fig.add_subplot(len(plot_variables)+1,len(sections_dict['lat']),(subplot_count+1)+(2*loopnum))
        obs_MHT=(obs_dict[section_choice]['MOCsig_MHTMFT'][y]/1e15).rolling(time=12).mean()
        obs_MHT['time']=obs_MHT['time']*30
        for keys in run_dict:
            (ComputeDiag_dict[section_choice][keys]['MOCsig_MHTMFT'][y]/1e15).resample(time="Y").mean().plot(alpha=.7,linewidth=.5,color=run_dict[keys]['color_identifier'])
            (ComputeDiag_dict[section_choice][keys]['MOCsig_MHTMFT'][y]/1e15).resample(time="Y").mean().rolling(time=smoothing_level, center=True).mean().plot(label=keys,color=run_dict[keys]['color_identifier'])
            # plt.plot((run_MHT.time-run_MHT.time[0])/360,run_MHT,label=keys,color=run_dict[keys]['color_identifier'])
        plt.title('ann. mean '+ComputeDiag_dict[section_choice][keys]['MOCsig_MHTMFT'][y].long_name+'. '+section_choice)
        plt.ylabel('PW')
        loopnum+=1
        plt.fill_between([np.min(start_dates),np.max(end_dates)],obs_MHT.mean()-obs_MHT.std(), obs_MHT.mean()+obs_MHT.std(), alpha=0.2, label='annual obs (mean+/-std')
        plt.xlabel('time')
    subplot_count+=1    

# Some additional comparison plots to observations, for a specified simulation and section choice (for all available variables, look at the descriptions of the output variables in SigmaZ_funcs.SigmaZ_diag_computeMFTMHT)

### Example 1: Model freshwater transports, plotted as a SigmaZ diagram:

In [ ]:
run_choice='odiv209'   # Choose a model
section_choice='sect2'  # choose one of the sections

In [ ]:
spec = gridspec.GridSpec(ncols=2, nrows=2,
                         width_ratios=[1,2], wspace=0.3,
                         hspace=0.3, height_ratios=[1, 2])
fig = plt.figure(figsize=(10,8))
ax0 = fig.add_subplot(spec[3])
ch1=ax0.pcolormesh(ComputeDiag_dict[section_choice][run_choice]['rho0_bounds'][0:-1],-rebin_depth,ComputeDiag_dict[section_choice][run_choice]['MOCsigz_MHTMFT'].Vso_z_rho.mean(axis=0)/1035/1e6,vmin=-2.5e-3,vmax=2.5e-3,cmap='RdBu_r')
ax0.set_title('Xsec FW transp. in Sigma-Z')
ax0.set_xlabel('Sigma (kg/m3)')
plt.xlim([26.5,28]), plt.ylim( [-4000,0] )
box = ax0.get_position()
ax0.set_position([box.x0*1, box.y0, box.width, box.height])
axColor = plt.axes([box.x0*1 + box.width * 1.05, box.y0, 0.01, box.height])
plt.colorbar(ch1, cax = axColor, orientation="vertical")
ax1 = fig.add_subplot(spec[1])
ax1.plot(ComputeDiag_dict[section_choice][run_choice]['rho0_ref'],ComputeDiag_dict[section_choice][run_choice]['MOCsig_MHTMFT'].MFT_zonmean_rho.mean(axis=0)/1035/1e6,label='TOTrho')
ax1.plot(ComputeDiag_dict[section_choice][run_choice]['rho0_ref'],ComputeDiag_dict[section_choice][run_choice]['MOCsig_MHTMFT'].MFT_MOCrho.mean(axis=0)/1035/1e6,label='MOCrho')
ax1.plot(ComputeDiag_dict[section_choice][run_choice]['rho0_ref'],ComputeDiag_dict[section_choice][run_choice]['MOCsig_MHTMFT'].MFT_GYRErho.mean(axis=0)/1035/1e6,label='GYRErho')
ax1.legend(loc='lower left')
ax1.set_title('Xsec FW transp. in Sigma')
ax1.set_ylabel('MFT (Sv)')
plt.xlim([26.5,28]), #plt.ylim( [-2,2] )
ax2 = fig.add_subplot(spec[2])
ax2.plot(ComputeDiag_dict[section_choice][run_choice]['MOCz_MHTMFT'].MFT_zonmean_z.mean(axis=0)/1035/1e6,-rebin_depth)
ax2.plot(ComputeDiag_dict[section_choice][run_choice]['MOCz_MHTMFT'].MFT_MOCz.mean(axis=0)/1035/1e6,-rebin_depth)
ax2.plot(ComputeDiag_dict[section_choice][run_choice]['MOCz_MHTMFT'].MFT_GYREz.mean(axis=0)/1035/1e6,-rebin_depth)
ax2.set_title('Xsec FW transp. in Z')
ax2.set_xlabel('MFT (Sv)')
ax2.set_ylabel('z (m)')    
plt.ylim([-4000,0]), #plt.xlim([-2,2]),

### Example 1 continued: Observed freshwater transports, plotted as a SigmaZ diagram:

In [ ]:
spec = gridspec.GridSpec(ncols=2, nrows=2,
                         width_ratios=[1,2], wspace=0.3,
                         hspace=0.3, height_ratios=[1, 2])
fig = plt.figure(figsize=(10,8))
ax0 = fig.add_subplot(spec[3])
ch1=ax0.pcolormesh(rho0_bounds[0:-1],-rebin_depth,obs_dict[section_choice]['MOCsigz_MHTMFT'].Vso_z_rho.mean(axis=0)/1035/1e6,vmin=-2.5e-3,vmax=2.5e-3,cmap='RdBu_r')
ax0.set_title('Xsec FW transp. in Sigma-Z')
ax0.set_xlabel('Sigma (kg/m3)')
plt.xlim([26.5,28]), plt.ylim( [-4000,0] )
box = ax0.get_position()
ax0.set_position([box.x0*1, box.y0, box.width, box.height])
axColor = plt.axes([box.x0*1 + box.width * 1.05, box.y0, 0.01, box.height])
plt.colorbar(ch1, cax = axColor, orientation="vertical")
ax1 = fig.add_subplot(spec[1])
ax1.plot(rho0_ref,obs_dict[section_choice]['MOCsig_MHTMFT'].MFT_zonmean_rho.mean(axis=0)/1035/1e6,label='TOTrho')
ax1.plot(rho0_ref,obs_dict[section_choice]['MOCsig_MHTMFT'].MFT_MOCrho.mean(axis=0)/1035/1e6,label='MOCrho')
ax1.plot(rho0_ref,obs_dict[section_choice]['MOCsig_MHTMFT'].MFT_GYRErho.mean(axis=0)/1035/1e6,label='GYRErho')
ax1.legend(loc='lower left')
ax1.set_title('Xsec FW transp. in Sigma')
ax1.set_ylabel('MFT (Sv)')
plt.xlim([26.5,28]), #plt.ylim( [-2,2] )
ax2 = fig.add_subplot(spec[2])
ax2.plot(obs_dict[section_choice]['MOCz_MHTMFT'].MFT_zonmean_z.mean(axis=0)/1035/1e6,-rebin_depth)
ax2.plot(obs_dict[section_choice]['MOCz_MHTMFT'].MFT_MOCz.mean(axis=0)/1035/1e6,-rebin_depth)
ax2.plot(obs_dict[section_choice]['MOCz_MHTMFT'].MFT_GYREz.mean(axis=0)/1035/1e6,-rebin_depth)
ax2.set_title('Xsec FW transp. in Z')
ax2.set_xlabel('MFT (Sv)')
ax2.set_ylabel('z (m)')    
plt.ylim([-4000,0]), #plt.xlim([-2,2]),

### Example 2: Model cross sections of V,T,S,rho:

In [ ]:
### Plot the property cross-sections. If calc_rho_flag=False then it will only show V and rho
if var_dict['numpy_algo']==False: 
    fig = plt.figure(figsize=(10,15))
    ax = fig.add_subplot(4,1,1)
    if var_dict['numpy_algo']: cell_area=cell_area[0,:,:]
    (ComputeDiag_dict[section_choice][run_choice]['transp'].mean('time')/ComputeDiag_dict[section_choice][run_choice]['cell_area']/1035).plot(ylim=[4000,0],levels=np.linspace(-.3,.3,num=31))
    ax.set_title(run_choice+' velocity across '+section_choice)
    ax.set_ylabel('Depth (m)'), ax.set_xlabel('')
    ax = fig.add_subplot(4,1,2)
    dens_maxAMOC=ComputeDiag_dict[section_choice][run_choice]['AMOCrho'].mean('time').idxmax('rho_bin')
    (ComputeDiag_dict[section_choice][run_choice]['rho'].mean('time')).plot(ylim=[4000,0],levels=np.linspace(26,28,num=41))
    (ComputeDiag_dict[section_choice][run_choice]['rho'].mean('time')).plot.contour(ylim=[4000,0],levels=1,vmin=dens_maxAMOC,vmax=dens_maxAMOC,colors=['red'])
    ax.set_title(run_choice+' Density along '+section_choice)
    ax.set_ylabel('Depth (m)'), ax.set_xlabel('')
    if var_dict['theta_var']:
        ax = fig.add_subplot(4,1,3)
        (ComputeDiag_dict[section_choice][run_choice]['thetao'].mean('time')).plot(ylim=[4000,0],levels=np.linspace(-1,12,num=32),cmap='viridis')
        ax.set_title(run_choice+' Temperature along '+section_choice)
        ax.set_ylabel('Depth (m)'), ax.set_xlabel('')
    if var_dict['salt_var']:
        ax = fig.add_subplot(4,1,4)
        (ComputeDiag_dict[section_choice][run_choice]['so'].mean('time')).plot(ylim=[4000,0],levels=np.linspace(34,35.5,num=31),cmap='viridis')
        ax.set_title(run_choice+' Salinity along '+section_choice)
        ax.set_ylabel('Depth (m)')

### Example 2 continued: Observed cross sections of V,T,S,rho:

In [ ]:
### Plot cross-sections of V, T, S and Rho along OSNAP East and West
fig = plt.figure(figsize=(10,15))
if numpy_algo: Area=Area[0,:,:]
ax = fig.add_subplot(4,1,1)
blah=ax.contourf(np.cumsum(obs_dict[section_choice]['section_gridwidth_OSNAP']),-depth_OSNAP,obs_dict[section_choice]['Transp_OSNAP'].mean('time')/obs_dict[section_choice]['Area'],levels=np.linspace(-.3,.3,num=31),extend='both',cmap='RdBu_r')
plt.colorbar(blah)
ax.set_title('velocity across OSNAP '+section_choice)
ax.set_ylabel('Depth (m)')
plt.ylim([-4000,0])
ax = fig.add_subplot(4,1,2)
vlah=ax.contourf(np.cumsum(obs_dict[section_choice]['section_gridwidth_OSNAP']),-depth_OSNAP,obs_dict[section_choice]['PD_OSNAP'].mean('time'),levels=np.linspace(26,28,num=41))
if numpy_algo==False: maxAMOC_contour=ax.contour(np.cumsum(obs_dict[section_choice]['section_gridwidth_OSNAP']),-depth_OSNAP,obs_dict[section_choice]['PD_OSNAP'].mean('time'),levels=[obs_dict[section_choice]['AMOCrho'].mean('time').idxmax('rho_bin')],colors=["red"])
plt.colorbar(vlah)
ax.set_title('Density along OSNAP '+section_choice)
ax.set_ylabel('Depth (m)')
plt.ylim([-4000,0])
ax = fig.add_subplot(4,1,3)
blah=ax.contourf(np.cumsum(obs_dict[section_choice]['section_gridwidth_OSNAP']),-depth_OSNAP,obs_dict[section_choice]['TEMP_OSNAP'].mean('time'),levels=np.linspace(-1,12,num=27))
plt.colorbar(blah)
ax.set_title('Temperature along OSNAP '+section_choice)
ax.set_ylabel('Depth (m)')
plt.ylim([-4000,0])
ax = fig.add_subplot(4,1,4)
blah=ax.contourf(np.cumsum(obs_dict[section_choice]['section_gridwidth_OSNAP']),-depth_OSNAP,obs_dict[section_choice]['PSAL_OSNAP'].mean('time'),levels=np.linspace(34,35.5,num=31),extend='both')
plt.colorbar(blah)
ax.set_title('Salinity along OSNAP '+section_choice)
ax.set_ylabel('Depth (m)')
ax.set_xlabel('Distance')
plt.ylim([-4000,0])

In [ ]:
# cluster.close()
# client.close()

In [ ]:
# Done! 